In [1]:
!pip install -U transformers datasets accelerate evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 738.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer,
    TrainingArguments,
)

import transformers


from datasets import (
    Dataset,
    load_dataset
)

import evaluate

import numpy as np

from tqdm import tqdm

In [3]:
transformers.logging.set_verbosity_info()

In [4]:
metric = evaluate.load("f1")

In [5]:
model_name = "FacebookAI/roberta-large"
dataset_name = "yahoo_answers_topics"

In [6]:
dataset = load_dataset(dataset_name, split="train[:20%]")

dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1400000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'topic', 'question_title', 'question_content', 'best_answer'],
    num_rows: 280000
})

In [7]:
label_names = dataset.features['topic'].names
id2label = dict(enumerate(label_names))
label2id = {label: i for i, label in id2label.items()}

In [8]:
label_names

['Society & Culture',
 'Science & Mathematics',
 'Health',
 'Education & Reference',
 'Computers & Internet',
 'Sports',
 'Business & Finance',
 'Entertainment & Music',
 'Family & Relationships',
 'Politics & Government']

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=len(label_names),
                                                           id2label=id2label,
                                                           label2id=label2id,)
collator = DataCollatorWithPadding(tokenizer)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--FacebookAI--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/config.json
Model config RobertaConfig {
  "_name_or_path": "FacebookAI/roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.39.3",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}



vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--FacebookAI--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--FacebookAI--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--FacebookAI--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--FacebookAI--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--FacebookAI--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/config.json
Model config RobertaConfig {
  "_

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

loading weights file model.safetensors from cache at /root/.cache/huggingface/hub/models--FacebookAI--roberta-large/snapshots/722cf37b1afa9454edce342e7895e588b6ff1d59/model.safetensors
Some weights of the model checkpoint at FacebookAI/roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initiali

In [10]:
def preprocess_dataset(examples):
  input = [f"{qt} {qc}? {ba}" for qt, qc, ba in zip(examples["question_title"], examples["question_content"], examples["best_answer"])]
  tokenized = tokenizer(input, truncation=True, padding=False)
  return {"input_ids": tokenized["input_ids"], "attention_mask": tokenized["attention_mask"], "labels": examples["topic"]}

In [11]:
tokenized_dataset = dataset.map(preprocess_dataset, batched=True, remove_columns=["id", "topic", "question_title", "question_content", "best_answer"])

tokenized_dataset

Map:   0%|          | 0/280000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 280000
})

In [12]:
tokenized_dataset[0]

{'input_ids': [0,
  25800,
  630,
  75,
  41,
  17547,
  18292,
  173,
  15,
  10,
  4049,
  2103,
  116,
  50,
  190,
  15,
  103,
  17408,
  28749,
  36132,
  15540,
  304,
  41,
  10918,
  8,
  10,
  2280,
  7,
  6042,
  5604,
  3156,
  9,
  5,
  4084,
  11352,
  5,
  18292,
  4,
  1437,
  20,
  4047,
  1075,
  1258,
  31,
  5,
  2280,
  16,
  13773,
  30,
  10,
  211,
  4186,
  36,
  30219,
  27695,
  42974,
  43,
  8,
  341,
  7,
  10933,
  27676,
  2485,
  11,
  5,
  7482,
  4084,
  8,
  3094,
  4298,
  4,
  993,
  3183,
  6,
  215,
  25,
  4049,
  6,
  20516,
  50,
  97,
  182,
  21199,
  6,
  8284,
  17408,
  15365,
  19,
  5,
  1460,
  9,
  5,
  211,
  4186,
  7,
  12775,
  11526,
  5,
  4084,
  11352,
  5,
  18292,
  4,
  1437,
  44128,
  282,
  11321,
  4049,
  16,
  8818,
  8,
  182,
  8284,
  6,
  5,
  18292,
  16,
  3276,
  7,
  1339,
  62,
  615,
  27676,
  2485,
  11,
  5,
  7482,
  4084,
  7,
  3094,
  4298,
  4,
  1437,
  6512,
  48723,
  17408,
  32,
  67,
  10,
  93

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [14]:
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir='models-roberta',
        evaluation_strategy='epoch',
        save_strategy='epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=2,
        weight_decay=0.01,
        metric_for_best_model='f1',
        load_best_model_at_end=True,
        report_to='none',
    ),
    train_dataset=tokenized_dataset,
    eval_dataset=load_dataset(dataset_name, split="test[:5%]").map(preprocess_dataset, batched=True, remove_columns=["id", "topic", "question_title", "question_content", "best_answer"]),
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

PyTorch: setting up devices


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 280,000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 70,000
  Number of trainable parameters = 355,369,994
